<a href="https://colab.research.google.com/github/ikechiamaka/WeatherPredictionLSTM_RNN/blob/main/Live_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math

# set up the API request
url = "http://api.openweathermap.org/data/2.5/forecast"
city_name = "Ottawa,CA"
api_key = "e6bc392482cd83b6d2d9da3c24f8f36f"
params = {"q": city_name, "appid": api_key, "units": "metric"}

# get the current time in the city
response = requests.get(url, params=params)
data = response.json()
city_timezone = timedelta(seconds=data["city"]["timezone"])
now = datetime.utcnow() + city_timezone

# get the weather data for the next 11 hours
weather_data = []
for i in range(11):
    # calculate the time for the current hour
    current_time = now + timedelta(hours=i)
    current_time_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print(f"Getting data for {current_time_str}...")
    
    # make the API request
    params = {"q": city_name, "appid": api_key, "units": "metric", "dt": int(current_time.timestamp())}
    response = requests.get(url, params=params)
    data = response.json()
    
    # get the temperature, pressure, and humidity
    temperature = data["list"][0]["main"]["temp"]
    pressure = data["list"][0]["main"]["pressure"]
    humidity = data["list"][0]["main"]["humidity"]
    
    # append the data to the list
    weather_data.append([current_time_str, temperature, pressure, humidity])

# create the dataframe
columns = ["time", "temperature", "pressure", "humidity"]
df = pd.DataFrame(weather_data, columns=columns)

# print the dataframe
print(df)


Getting data for 2023-03-10 09:09:01...
Getting data for 2023-03-10 10:09:01...
Getting data for 2023-03-10 11:09:01...
Getting data for 2023-03-10 12:09:01...
Getting data for 2023-03-10 13:09:01...
Getting data for 2023-03-10 14:09:01...
Getting data for 2023-03-10 15:09:01...
Getting data for 2023-03-10 16:09:01...
Getting data for 2023-03-10 17:09:01...
Getting data for 2023-03-10 18:09:01...
Getting data for 2023-03-10 19:09:01...
                   time  temperature  pressure  humidity
0   2023-03-10 09:09:01        -3.67      1023        74
1   2023-03-10 10:09:01        -3.63      1024        74
2   2023-03-10 11:09:01        -3.63      1024        74
3   2023-03-10 12:09:01        -3.63      1024        74
4   2023-03-10 13:09:01        -3.63      1024        74
5   2023-03-10 14:09:01        -3.67      1023        74
6   2023-03-10 15:09:01        -3.63      1024        74
7   2023-03-10 16:09:01        -3.63      1024        74
8   2023-03-10 17:09:01        -3.65      1024 

In [58]:
tx_power = 20 # dBm
rx_sensitivity = -90 # dBm
frequency = 2.4 # GHz
c = 3e8 # m/s
lambda_ = c / (frequency * 1e9) # wavelength in meters
tx_antenna_gain = 10 # dBi
rx_antenna_gain = 10 # dBi
distance = 1000 # meters

# Define a function to calculate the free space path loss
def calculate_friis_loss(distance, frequency, tx_antenna_gain, rx_antenna_gain):
    gain = tx_antenna_gain + rx_antenna_gain
    return 20 * math.log10(distance) + 20 * math.log10(frequency) - 147.55 - gain

# Define a function to calculate the weather loss
def calculate_weather_loss(temperature, humidity, pressure):
    vapor_pressure = humidity * pressure / 100
    saturation_vapor_pressure = 6.11 * 10 ** (7.5 * temperature / (237.3 + temperature))
    water_vapor_density = 1000 * vapor_pressure / (287.058 * (temperature + 273.15))
    attenuation = 0.0172 * water_vapor_density * (1 + 0.0004 * frequency) / (math.sin(0.5 * lambda_)) ** 0.5
    return attenuation * distance / 1000


In [59]:
# Initialize an empty list to store the SNR, BER, capacity, and bandwidth values
snr_ber_data = []
threshold_snr = 1e-10 # set a threshold SNR value below which the BER is set to 0.5 (random guessing)
bandwidth = 10e6  # set the bandwidth to 10 MHz

# Iterate over the weather data and calculate the SNR, BER, capacity, and bandwidth for each combination of weather conditions
for index, row in df.iterrows():
    # Get the weather conditions for this row
    temperature = row["temperature"]
    humidity = row["humidity"]
    pressure = row["pressure"]
    date_time = row["time"] # assuming there is a "datetime" column in the weather_data DataFrame

    # Calculate the free space path loss
    friis_loss = calculate_friis_loss(distance, frequency, tx_antenna_gain, rx_antenna_gain)

    # Calculate the weather loss
    weather_loss = calculate_weather_loss(temperature, humidity, pressure)

    # Calculate the SNR
    snr = tx_power - rx_sensitivity - friis_loss - weather_loss

    # Check if the SNR is negative
    if snr < threshold_snr:
        ber = 0.5
    else:
        # Calculate the BER for 0QPSK modulation
        ber = 0.5 * math.erfc(math.sqrt(snr))

    # Calculate the capacity
    capacity = bandwidth * (np.log2(1 + snr / 3.01))

    # Append the SNR, BER, capacity, and bandwidth values to the list
    snr_ber_data.append({
        "datetime": date_time,
        "temperature": temperature,
        "humidity": humidity,
        "pressure": pressure,
        "snr": snr,
        "ber": ber,
        "capacity": capacity,
        "bandwidth": bandwidth
    })

# Create a DataFrame from the list and set the datetime column as the index
snr_ber_data = pd.DataFrame(snr_ber_data)
snr_ber_data.set_index("datetime", inplace=True)


In [60]:
snr_ber_data

,temperature,humidity,pressure,snr,ber,capacity,bandwidth
datetime,,,,,,,
2023-03-10 09:09:01,-3.67,74,1023,209.271623,2.532195e-93,6.140072e+07,10000000.0
2023-03-10 10:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 11:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 12:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 13:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 14:09:01,-3.67,74,1023,209.271623,2.532195e-93,6.140072e+07,10000000.0
2023-03-10 15:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 16:09:01,-3.63,74,1024,209.271065,2.533614e-93,6.140068e+07,10000000.0
2023-03-10 17:09:01,-3.65,74,1024,209.271014,2.533741e-93,6.140068e+07,10000000.0


In [61]:
snr_ber_data.index = pd.to_datetime(snr_ber_data.index)
snr_ber_data.index

DatetimeIndex(['2023-03-10 09:09:01', '2023-03-10 10:09:01',
               '2023-03-10 11:09:01', '2023-03-10 12:09:01',
               '2023-03-10 13:09:01', '2023-03-10 14:09:01',
               '2023-03-10 15:09:01', '2023-03-10 16:09:01',
               '2023-03-10 17:09:01', '2023-03-10 18:09:01',
               '2023-03-10 19:09:01'],
              dtype='datetime64[ns]', name='datetime', freq=None)

In [62]:
snr_ber_data.index


DatetimeIndex(['2023-03-10 09:09:01', '2023-03-10 10:09:01',
               '2023-03-10 11:09:01', '2023-03-10 12:09:01',
               '2023-03-10 13:09:01', '2023-03-10 14:09:01',
               '2023-03-10 15:09:01', '2023-03-10 16:09:01',
               '2023-03-10 17:09:01', '2023-03-10 18:09:01',
               '2023-03-10 19:09:01'],
              dtype='datetime64[ns]', name='datetime', freq=None)

In [63]:
temperature = snr_ber_data['temperature']

In [64]:
# Create a new DataFrame called temp_df with a single column 'Temperature' containing the values from 'temp'
Actual_df = pd.DataFrame({'Temperature': temperature})

# # Map the index of temp_df to seconds and add a new column called 'Seconds'
Actual_df['Seconds'] = Actual_df.index.map(pd.Timestamp.timestamp)


In [65]:
# Define the number of seconds in a day and a year based on the number of seconds in an hour, day, and year
day = 60 * 60 * 24
year = 365.2425 * day

# Add four new columns to Actual_df called 'Day sin', 'Day cos', 'Year sin', and 'Year cos' that represent the cyclic nature of time
Actual_df['Day sin'] = np.sin(Actual_df['Seconds'] * (2 * np.pi / day))
Actual_df['Day cos'] = np.cos(Actual_df['Seconds'] * (2 * np.pi / day))
Actual_df['Year sin'] = np.sin(Actual_df['Seconds'] * (2 * np.pi / year))
Actual_df['Year cos'] = np.cos(Actual_df['Seconds'] * (2 * np.pi / year))


# Drop the 'Seconds' column from the DataFrame temp_df
Actual_df = Actual_df.drop('Seconds', axis=1)

# Display the first few rows of the modified DataFrame temp_df
Actual_df.head()


,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,
2023-03-10 09:09:01,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [66]:
df = pd.concat([snr_ber_data['pressure'], Actual_df], axis=1)
df.head()

,pressure,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,,
2023-03-10 09:09:01,1023,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,1024,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,1024,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,1024,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,1024,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [67]:
df = pd.concat([snr_ber_data['humidity'], df], axis=1)
df.head()

,humidity,pressure,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,,,
2023-03-10 09:09:01,74,1023,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,74,1024,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,74,1024,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,74,1024,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,74,1024,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [68]:
df = pd.concat([snr_ber_data['snr'], df], axis=1)
df.head()

,snr,humidity,pressure,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,,,,
2023-03-10 09:09:01,209.271623,74,1023,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,209.271065,74,1024,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,209.271065,74,1024,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,209.271065,74,1024,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,209.271065,74,1024,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [69]:
df = pd.concat([snr_ber_data['ber'], df], axis=1)
df.head()

,ber,snr,humidity,pressure,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,,,,,
2023-03-10 09:09:01,2.532195e-93,209.271623,74,1023,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,2.533614e-93,209.271065,74,1024,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,2.533614e-93,209.271065,74,1024,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,2.533614e-93,209.271065,74,1024,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,2.533614e-93,209.271065,74,1024,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [70]:
df = pd.concat([snr_ber_data['capacity'], df], axis=1)
df.head()

,capacity,ber,snr,humidity,pressure,Temperature,Day sin,Day cos,Year sin,Year cos
datetime,,,,,,,,,,
2023-03-10 09:09:01,6.140072e+07,2.532195e-93,209.271623,74,1023,-3.67,0.678747,-0.734372,0.924181,0.381956
2023-03-10 10:09:01,6.140068e+07,2.533614e-93,209.271065,74,1024,-3.63,0.465550,-0.885021,0.924454,0.381293
2023-03-10 11:09:01,6.140068e+07,2.533614e-93,209.271065,74,1024,-3.63,0.220627,-0.975358,0.924727,0.380630
2023-03-10 12:09:01,6.140068e+07,2.533614e-93,209.271065,74,1024,-3.63,-0.039332,-0.999226,0.925000,0.379967
2023-03-10 13:09:01,6.140068e+07,2.533614e-93,209.271065,74,1024,-3.63,-0.296611,-0.954998,0.925272,0.379304


In [71]:
df.shape

(11, 10)

In [72]:
df_to_np = df.to_numpy()

In [73]:
# Reshape live_data to have 11 time steps
live_data =df_to_np.reshape(1, 11, 10)

In [74]:
# Calculate mean and standard deviation for each feature in the input set
cap_mean = np.mean(live_data[:, :, 0])
cap_std = np.std(live_data[:, :, 0])

ber_mean = np.mean(live_data[:, :, 1])
ber_std = np.std(live_data[:, :, 1])

snr_mean = np.mean(live_data[:, :, 2])
snr_std = np.std(live_data[:, :, 2])

h_mean = np.mean(live_data[:, :, 3])
h_std = np.std(live_data[:, :, 3])

p_mean = np.mean(live_data[:, :, 4])
p_std = np.std(live_data[:, :, 4])

temp_mean = np.mean(live_data[:, :, 5])
temp_std = np.std(live_data[:, :, 5])

In [75]:
from keras.models import load_model

# Load the trained model
model5 = load_model('/content/drive/MyDrive/model5')
model5.output.shape

TensorShape([None, 6])

In [76]:
# Generate predictions for the live data
predictions = model5.predict(live_data)


cap_preds, ber_preds, snr_preds, h_preds, p_preds, temp_preds = predictions[:, 0], predictions[:, 1], predictions[:, 2], predictions[:, 3], predictions[:, 4], predictions[:, 5]

# Create a DataFrame to store the predictions and actual value
df = pd.DataFrame(data={'Temperature Predictions': temp_preds,
                            
                            'Pressure Predictions': p_preds,
                            
                            'Humidity Predictions': h_preds,
                            
                            'Capacity Predictions': cap_preds,
                           
                            'BER Predictions': ber_preds,
                            
                            'SNR Predictions': snr_preds,
                            



                            })

1/1 [==============================] - 0s 467ms/step


In [77]:
df

,Temperature Predictions,Pressure Predictions,Humidity Predictions,Capacity Predictions,BER Predictions,SNR Predictions
0,0.004074,0.299511,0.023104,0.128832,-0.48898,0.635659


In [78]:
def postprocess_temp(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*temp_std) + temp_mean
  return arr

def postprocess_p(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*p_std) + p_mean
  return arr

def postprocess_h(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*h_std) + h_mean
  return arr

def postprocess_cap(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*cap_std) + cap_mean
  return arr

def postprocess_snr(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*snr_std) + snr_mean
  return arr

def postprocess_ber(arr):
  # multiply array by standard deviation and add mean to reverse normalization
  arr = (arr*ber_std) + ber_mean
  return arr

In [79]:
cap_preds, ber_preds, snr_preds, h_preds, p_preds, temp_preds = postprocess_cap(predictions[:, 0]), postprocess_ber(predictions[:, 1]),  postprocess_snr(predictions[:, 2]), postprocess_h(predictions[:, 3]), postprocess_p(predictions[:, 4]), postprocess_temp(predictions[:, 5])
# Create a DataFrame to store the predictions and actual value
df = pd.DataFrame(data={'Temperature Predictions': temp_preds,
                            
                            'Pressure Predictions': p_preds,
                            
                            'Humidity Predictions': h_preds,
                            
                            'Capacity Predictions': cap_preds,
                           
                            'BER Predictions': ber_preds,
                            
                            'SNR Predictions': snr_preds,
                            



                            })

In [80]:
df

,Temperature Predictions,Pressure Predictions,Humidity Predictions,Capacity Predictions,BER Predictions,SNR Predictions
0,-3.642656,1023.860657,74.0,61400696.0,0.0,209.271378
